# ChatGPT'nin optimize ettigi path ve file yazan kod

In [47]:
import re
import os

def extract_and_write_data(directory_path, output_file):
    pattern = r"\.\.\.Telekom.*?\s{2}"
    total_files = 0
    processed_files = 0
    unprocessed_files = []

    def process_file(filename):
        nonlocal processed_files
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as source:
                content = source.read()
                match = re.search(pattern, content, re.DOTALL)
                if match:
                    processed_files += 1
                    matched_text = match.group(0)
                    cleaned_text = matched_text.strip(".").strip()
                    sections = re.split(r'\.\.\.\.+', cleaned_text)
                    sections = [section.strip() for section in sections if section.strip()]
                    if sections and sections[-1].endswith("..."):
                        sections[-1] = sections[-1].rstrip(".")
                    return filename, sections
                else:
                    return filename, None
        except Exception as e:
            return filename, str(e)

    with open(output_file, "w", encoding="utf-8") as output:
        for filename in os.listdir(directory_path):
            if filename.endswith(".txt"):
                total_files += 1
                result = process_file(filename)
                if isinstance(result[1], str):
                    unprocessed_files.append(f"{result[0]}: Error - {result[1]}")
                elif result[1] is None:
                    unprocessed_files.append(f"{result[0]}: No matching content found.")
                else:
                    output.write(f"File: {result[0]}\n")
                    for section in result[1]:
                        output.write(f"{section}\n")
                    output.write("\n" + "="*40 + "\n")

        output.write(f"\nTotal number of .txt files in the folder: {total_files}\n")
        output.write(f"Number of .txt files processed and written to data.txt: {processed_files}\n")
        if unprocessed_files:
            output.write("\nUnprocessed files:\n")
            for entry in unprocessed_files:
                output.write(f"{entry}\n")

    print("Process completed. The extracted sections from all text files have been written to data.txt.")
    print(f"Total number of .txt files in the folder: {total_files}")
    print(f"Number of .txt files processed and written to data.txt: {processed_files}")
    print(f"Unprocessed files and reasons have been written to data.txt.")

# Kullanım
directory_path = "/Users/taha/Desktop/scrapeV2/output_folder"
output_file = "data.txt"
extract_and_write_data(directory_path, output_file)


Process completed. The extracted sections from all text files have been written to data.txt.
Total number of .txt files in the folder: 2595
Number of .txt files processed and written to data.txt: 1676
Unprocessed files and reasons have been written to data.txt.


# data.txt ve data.csv - navigation soru ve cevap - ayiklanmadi ama

In [56]:
import re
import os
import csv

def extract_and_write_data(directory_path, output_file_txt, output_file_csv):
    # Regex pattern: Start with "...Telekom" and end with two spaces
    pattern = r"\.\.\.Telekom.*?\s{2}"
    
    # Initialize counters and lists
    total_files = 0
    processed_files = 0
    unprocessed_files = []

    def process_file(filename):
        nonlocal processed_files
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as source:
                content = source.read()
                match = re.search(pattern, content, re.DOTALL)
                if match:
                    processed_files += 1
                    matched_text = match.group(0)
                    start_index = match.end()

                    # Clean and format the matched text
                    # Remove leading and trailing dots and split by dots
                    cleaned_text = re.sub(r'\.\.\.+', '\n', matched_text.strip(".").strip())
                    
                    # Extract text after the pattern
                    post_pattern_text = content[start_index:]
                    
                    # Regex pattern to find paragraphs separated by multiple newlines
                    paragraph_pattern = r'([^\n]+(?:\n[^\n]+)*)(?:\n{2,})'
                    
                    paragraphs = re.findall(paragraph_pattern, post_pattern_text)
                    
                    if len(paragraphs) >= 2:
                        # Extract the first and second paragraphs
                        first_paragraph = paragraphs[0].strip()
                        second_paragraph = paragraphs[1].strip()
                        
                        # Check for excessive newlines after the second paragraph
                        if len(paragraphs) > 2:
                            # Find the number of newlines after the second paragraph
                            remaining_text = post_pattern_text[post_pattern_text.find(paragraphs[1]) + len(paragraphs[1]):]
                            newlines_after_second = re.match(r'\n{5,}', remaining_text)
                            if newlines_after_second:
                                return filename, (cleaned_text, first_paragraph, second_paragraph)
                        else:
                            return filename, (cleaned_text, first_paragraph, second_paragraph)
                return filename, None
        except Exception as e:
            return filename, str(e)

    # Write to both TXT and CSV files
    with open(output_file_txt, "w", encoding="utf-8") as output_txt, \
         open(output_file_csv, "w", newline='', encoding="utf-8") as output_csv:
        
        csv_writer = csv.writer(output_csv)
        csv_writer.writerow(["File", "Navigation", "First Paragraph", "Second Paragraph"])
        
        for filename in os.listdir(directory_path):
            if filename.endswith(".txt"):
                total_files += 1
                result = process_file(filename)
                if isinstance(result[1], str):
                    unprocessed_files.append(f"{result[0]}: Error - {result[1]}")
                elif result[1] is None:
                    unprocessed_files.append(f"{result[0]}: No matching content found.")
                else:
                    navigation, first_paragraph, second_paragraph = result[1]
                    
                    # Write to TXT file
                    output_txt.write(f"File: {filename}\n")
                    output_txt.write(f"Navigation:\n{navigation}\n\n")
                    output_txt.write(f"First Paragraph:\n{first_paragraph}\n\n")
                    output_txt.write(f"Second Paragraph:\n{second_paragraph}\n")
                    output_txt.write("\n" + "="*40 + "\n")
                    
                    # Write to CSV file
                    csv_writer.writerow([filename, navigation.replace('\n', ' '), first_paragraph.replace('\n', ' '), second_paragraph.replace('\n', ' ')])

        # Write summary information to TXT file
        output_txt.write(f"\nTotal number of .txt files in the folder: {total_files}\n")
        output_txt.write(f"Number of .txt files processed and written to data.txt: {processed_files}\n")
        if unprocessed_files:
            output_txt.write("\nUnprocessed files:\n")
            for entry in unprocessed_files:
                output_txt.write(f"{entry}\n")

    print("Process completed. The extracted sections from all text files have been written to data.txt and data.csv.")
    print(f"Total number of .txt files in the folder: {total_files}")
    print(f"Number of .txt files processed and written to data.txt: {processed_files}")
    print(f"Unprocessed files and reasons have been written to data.txt.")

# Kullanım
directory_path = "/Users/taha/Desktop/scrapeV2/output_folder"
output_file_txt = "data.txt"
output_file_csv = "data.csv"
extract_and_write_data(directory_path, output_file_txt, output_file_csv)


Process completed. The extracted sections from all text files have been written to data.txt and data.csv.
Total number of .txt files in the folder: 2595
Number of .txt files processed and written to data.txt: 1676
Unprocessed files and reasons have been written to data.txt.
